In [156]:
from finvizfinance.screener.overview import Overview
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
import certifi
import json
from urllib.request import urlopen
warnings.filterwarnings("ignore")
from datetime import datetime
today = datetime.today().strftime('%Y_%m_%d')

In [130]:
stock_scores = pd.read_csv('stock_scores/stock_score_data_recent.csv')

## Get Value Stocks

In [131]:
filters_dict = {'Debt/Equity':'Under 1',
                    'Operating Margin':'Positive (>0%)', 
                    'P/E':'Under 20',
                    'InsiderTransactions':'Positive (>0%)'}

In [132]:
foverview = Overview()
foverview.set_filter(filters_dict=filters_dict)
df_overview = foverview.screener_view()
tickers = df_overview['Ticker'].to_list()

In [133]:
url = 'https://valueinvestorsclub.com/ideas'
def get_trending_symbols(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve content: {response.status_code}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the <b>TRENDING:</b> element
    trending_header = soup.find('b', string='TRENDING:')

    if not trending_header:
        print("No trending section found.")
        return []

    # Get the next sibling which should contain the <span> with <a> tags
    trending_span = trending_header.find_next('span')
    
    # If we found the span, extract the symbols
    symbols = []
    if trending_span:
        # Find all <a> tags with href starting with '/idea/'
        for a_tag in trending_span.find_all('a', href=lambda href: href and href.startswith('/idea/')):
            symbols.append(a_tag.text.strip())

    return symbols

# Example usage
trending_symbols = get_trending_symbols(url)

In [134]:
value_stocks = tickers + trending_symbols
value_stocks = list(set(value_stocks))

## Filter to value stocks

In [135]:
value_scores = stock_scores[stock_scores['ticker'].isin(value_stocks)]
value_scores = value_scores[['ticker', '2023', 'roce_rank', 'coef_rank', 'std_rank', 'final_rank']]

In [136]:
value_scores.sort_values(by='final_rank').head(5)

,ticker,2023,roce_rank,coef_rank,std_rank,final_rank
554,AEHR,0.189696,313.0,48.0,2251.0,42.0
1635,MLI,0.337753,173.0,147.0,1838.0,49.0
624,AMRK,0.375883,231.0,403.0,1817.0,168.5
1485,KMT,0.092343,436.0,323.0,1450.0,177.0
495,XOM,0.170580,643.0,383.0,1414.0,258.0


## Filter to stocks that are trending

In [137]:
#Need to make this filter more restrictive, then go into the momentem algorithm
filters_dict = {'20-Day Simple Moving Average': 'Price above SMA20', 
                '50-Day Simple Moving Average' : 'SMA50 below SMA20',
                #'200-Day Simple Moving Average': 'SMA200 below SMA50',
               "Index": 'Any' ,
               '20-Day High/Low': '0-5% below High'} #'RUSSELL 2000'
foverview = Overview()
foverview.set_filter(filters_dict=filters_dict)
df_overview = foverview.screener_view()
trending_stocks = df_overview['Ticker'].to_list()

In [138]:
final_stocks = value_scores[value_scores['ticker'].isin(trending_stocks)].sort_values(by='final_rank')

In [171]:
investing_stocks = final_stocks[final_stocks['final_rank'] <= 200]

#get current price of stocks
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)
    
def get_price(tick):
    url = (f"https://financialmodelingprep.com/api/v3/quote/{tick}?apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
    data = get_jsonparsed_data(url)
    price = data[0]['price']
    return price

investing_stocks['price_picked'] = investing_stocks['ticker'].apply(get_price)

print(investing_stocks)

     ticker      2023  roce_rank  coef_rank  std_rank  final_rank        date  \
1635    MLI  0.337753      173.0      147.0    1838.0        49.0  2024_10_28   

      price_picked  
1635         81.85  


In [172]:
investing_stocks.to_csv('investing_stocks.csv', mode='a', header=False, index=False)

In [161]:
#get current price of stocks
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)
    
def get_price(tick):
    url = (f"https://financialmodelingprep.com/api/v3/quote/{tick}?apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
    data = get_jsonparsed_data(url)
    price = data[0]['price']
    return price

investing_stocks['price_picked'] = investing_stocks['ticker'].apply(get_price)